# pyDatalog通过知识推理可以实现的图算法
## 在图、树、层次结构均可使用
##### 可以从一个节点到达别的哪些节点
##### 两个节点之间可能的路径是什么
##### 以更低的时间复杂度给出两个节点之间的一条路径
##### 给定代价函数，两个节点之间的最短路径

In [1]:
from pyDatalog import pyDatalog
#            4
#           /
#    1 -  2 -  3
#       /  \
#      7    5
#      \   /
#        6     8-9
pyDatalog.create_terms('can_reach,X,Y,Z,link,shortest_path')
pyDatalog.create_terms('all_path,P,P2,path,safe_path,path_with_cost,C,C2')
+link(1,2)
+link(2,3)
+link(2,4)
+link(2,5)
+link(5,6)
+link(6,7)
+link(7,2)
+link(8,9)

In [2]:
#无向图双向关系
link(X,Y) <= link(Y,X)
print (link(1,Y))

Y
-
2


In [3]:
print("can reach from 1")
can_reach(X,Y) <= can_reach(X,Z) & link(Z,Y) & (X!=Y)
can_reach(X,Y) <= link(X,Y)
print (can_reach(1,Y))

can reach from 1
Y
-
2
3
4
5
6
7


In [4]:
print("can't reach from 1")
print(link(X,Y) & (~can_reach(1,X)) & (X !=1))

can't reach from 1
X | Y
--|--
9 | 8
8 | 9


In [5]:
print("all path from/to 1")
#能够link的节点即意味着可达，记录通过的节点
all_path(X,Y,P) <= all_path(X,Z,P2) & link(Z,Y) & (X!=Y) & (X._not_in(P2)) & (Y._not_in(P2)) & (P==P2+[Z])
all_path(X,Y,P) <= link(X,Y) & (P==[])
print (all_path(X,1,P))

all path from/to 1
X | P        
--|----------
2 | ()       
7 | (2,)     
5 | (2,)     
4 | (2,)     
3 | (2,)     
6 | (7, 2)   
6 | (5, 2)   
5 | (6, 7, 2)
7 | (6, 5, 2)


In [6]:
print("no path from 1")
print(link(X,Y) & (~all_path(1,X,P)) & (X !=1))

no path from 1
X | Y
--|--
9 | 8
8 | 9


In [8]:
print("a path from / to 1")
(path[X,Y]==P) <= ((path[X,Z]==P2) &  link(Z,Y)  & (P==P2+[Z])) 
# 下面这行的注释应该会存在问题，是不安全的优化，会存在X和Y相同的情况
# & (X!=Y) & (X._not_in(P2)) & (Y._not_in(P2)) 
#                    & (P==P2+[Z])) 
(path[X,Y]==P) <= link(X,Y) & (P==[])

print (path[1,Y]==P)

a path from / to 1
Y | P     
--|-------
2 | ()    
4 | (2,)  
5 | (2,)  
3 | (2,)  
6 | (2, 5)
7 | (2,)  
1 | (2,)  


In [9]:
#加上了X不等于Y的条件
print("not one path from 1")
(safe_path[X,Y]==P) <= ((safe_path[X,Z]==P2) &  link(Z,Y)
                   # next line needed to avoid infinite loop in negation
                   & (X!=Y) & (X._not_in(P2)) & (Y._not_in(P2)) 
                   & (P==P2+[Z])) 
(safe_path[X,Y]==P) <= link(X,Y) & (P==[])
print(link(X,Y) & (X !=1) & (~(safe_path[X,1]==P)) )

not one path from 1
X | Y
--|--
9 | 8
8 | 9


In [11]:
print ("path with cost from / to 1")
(path_with_cost(X,Y,P,C)) <= (path_with_cost(X,Z,P2,C2)) & link(Z,Y) & (X!=Y) & (X._not_in(P2)) & (Y._not_in(P2)) & (P==P2+[Z]) & (C==C2+1) 
(path_with_cost(X,Y,P,C)) <= link(X,Y) & (P==[]) & (C==0)
print (path_with_cost(1,Y,P,C))
print (path_with_cost(Y,1,P,C))


path with cost from / to 1
Y | P         | C
--|-----------|--
2 | ()        | 0
3 | (2,)      | 1
5 | (2,)      | 1
4 | (2,)      | 1
7 | (2,)      | 1
6 | (2, 5)    | 2
6 | (2, 7)    | 2
7 | (2, 5, 6) | 3
5 | (2, 7, 6) | 3
Y | P         | C
--|-----------|--
2 | ()        | 0
7 | (2,)      | 1
5 | (2,)      | 1
4 | (2,)      | 1
3 | (2,)      | 1
6 | (7, 2)    | 2
6 | (5, 2)    | 2
5 | (6, 7, 2) | 3
7 | (6, 5, 2) | 3


In [12]:
print ("shortest path from / to 1")
(shortest_path[X,Y]==min_(P, order_by=C)) <= (path_with_cost(X,Y,P,C))

print (shortest_path[1,Y]==P) 
print (shortest_path[X,1]==P)

shortest path from / to 1
Y | P     
--|-------
7 | (2,)  
6 | (2, 5)
5 | (2,)  
4 | (2,)  
3 | (2,)  
2 | ()    
X | P     
--|-------
7 | (2,)  
6 | (7, 2)
5 | (2,)  
4 | (2,)  
3 | (2,)  
2 | ()    
